In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# =============================================================================
# STEP 1: Download and Read the Public MET Data
# =============================================================================
# Make sure that data_url points to the actual CMS Open Data Portal MET data file (in CSV format)
data_url = 'https://opendata.cern.ch/record/1234567/files/cms_met_data.csv'  # <-- Replace with the actual URL

# First, try to read the file locally; if not found, download it
try:
    df = pd.read_csv("cms_met_data.csv")
    print("Data loaded from local file 'cms_met_data.csv'.")
except FileNotFoundError:
    print("Local file not found, downloading data...")
    response = requests.get(data_url)
    if response.status_code == 200:
        with open("cms_met_data.csv", "wb") as f:
            f.write(response.content)
        df = pd.read_csv("cms_met_data.csv")
        print("Data downloaded and saved to 'cms_met_data.csv'.")
    else:
        raise Exception(f"Download failed, HTTP status code: {response.status_code}")

# =============================================================================
# STEP 1.1: Data Cleaning and Missing Value Check
# =============================================================================
# Check if the "MET" column exists
if "MET" not in df.columns:
    raise ValueError("The 'MET' column was not found in the data. Please check the file format or column name.")

# Remove rows with missing or non-numeric values in the MET column
df = df[pd.to_numeric(df["MET"], errors='coerce').notnull()]
df["MET"] = pd.to_numeric(df["MET"])  # Convert to numeric type

# Add any other necessary filtering (e.g., based on trigger conditions or event quality)
# For example: df = df[df["quality_flag"] == 1]

print(f"After cleaning, there are {len(df)} MET records in the data.")

# =============================================================================
# STEP 2: Data Preprocessing: Create Histogram and Estimate Statistical Errors
# =============================================================================
# Define the binning for the histogram: here using 50 equally spaced bins from 0 to 500 GeV (adjust as needed)
bins = np.linspace(0, 500, 51)
counts, bin_edges = np.histogram(df["MET"].values, bins=bins)
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
bin_width = bin_edges[1] - bin_edges[0]

# Check the event count in each bin; if some bins have very few events, print a warning
low_count_bins = np.where(counts < 5)[0]
if len(low_count_bins) > 0:
    print(f"Warning: The following bins have few events: {low_count_bins.tolist()}. Please check the binning settings.")

# Convert the histogram to a probability density by normalizing to the total number of events and bin width
n_events = len(df["MET"].values)
density = counts / (n_events * bin_width)

# Estimate the statistical (Poisson) error for each bin
errors = np.sqrt(counts) / (n_events * bin_width)

# =============================================================================
# STEP 3: Define Three Theoretical Models (ΛCDM, MOND, Quaternionic Model)
# =============================================================================
def model_LCDM(ET, E0):
    """
    ΛCDM Model:
    Assumes the MET distribution follows an exponential distribution:
    P(ET) = (1/E0) * exp(-ET/E0)
    """
    return (1.0 / E0) * np.exp(-ET / E0)

def model_MOND(ET, E0):
    """
    MOND Model (example):
    Uses a modified power-law distribution:
    P(ET) = (1/E0) * (1 + ET/E0)^(-2)
    """
    return (1.0 / E0) * (1 + ET / E0)**(-2)

def model_quaternionic(ET, E0, alpha):
    """
    Quaternionic Model (PT-Symmetric Quaternionic Spacetime):
    The modified distribution is defined as:
      P(ET) = (1/E0) * exp(-ET/E0) * [1 + alpha*sin^2(ET/E0)] / norm
    where norm = 1 + 0.4*alpha is a normalization constant ensuring the PDF integrates to 1,
    and alpha represents the strength of the additional imaginary effect.
    """
    norm = 1 + 0.4 * alpha  # This normalization constant can be further adjusted based on theoretical derivation
    return (1.0 / E0) * np.exp(-ET / E0) * (1 + alpha * np.sin(ET / E0)**2) / norm

# =============================================================================
# STEP 4: Use curve_fit to Fit Each Model
# =============================================================================
# --- Fit the ΛCDM Model (with one free parameter E0) ---
popt_LCDM, pcov_LCDM = curve_fit(model_LCDM, bin_centers, density, p0=[80], sigma=errors, absolute_sigma=True)
fitted_E0_LCDM = popt_LCDM[0]
# Print the uncertainties of the fitted parameter
perr_LCDM = np.sqrt(np.diag(pcov_LCDM))
print("\n=== ΛCDM Model Fit Results ===")
print(f"Fitted E0 = {fitted_E0_LCDM:.2f} ± {perr_LCDM[0]:.2f} GeV")
# Calculate the model values and chi2, etc.
model_LCDM_vals = model_LCDM(bin_centers, fitted_E0_LCDM)
chi2_LCDM = np.sum(((density - model_LCDM_vals) / errors)**2)
dof_LCDM = len(bin_centers) - 1  # 1 free parameter
AIC_LCDM = chi2_LCDM + 2 * 1
BIC_LCDM = chi2_LCDM + 1 * np.log(len(bin_centers))
print(f"chi2 = {chi2_LCDM:.2f}, degrees of freedom = {dof_LCDM}, AIC = {AIC_LCDM:.2f}, BIC = {BIC_LCDM:.2f}")

# --- Fit the MOND Model (with one free parameter E0) ---
popt_MOND, pcov_MOND = curve_fit(model_MOND, bin_centers, density, p0=[80], sigma=errors, absolute_sigma=True)
fitted_E0_MOND = popt_MOND[0]
perr_MOND = np.sqrt(np.diag(pcov_MOND))
print("\n=== MOND Model Fit Results ===")
print(f"Fitted E0 = {fitted_E0_MOND:.2f} ± {perr_MOND[0]:.2f} GeV")
model_MOND_vals = model_MOND(bin_centers, fitted_E0_MOND)
chi2_MOND = np.sum(((density - model_MOND_vals) / errors)**2)
dof_MOND = len(bin_centers) - 1
AIC_MOND = chi2_MOND + 2 * 1
BIC_MOND = chi2_MOND + 1 * np.log(len(bin_centers))
print(f"chi2 = {chi2_MOND:.2f}, degrees of freedom = {dof_MOND}, AIC = {AIC_MOND:.2f}, BIC = {BIC_MOND:.2f}")

# --- Fit the Quaternionic Model (with two free parameters: E0 and alpha) ---
popt_quat, pcov_quat = curve_fit(model_quaternionic, bin_centers, density, p0=[80, 0.3], sigma=errors, absolute_sigma=True)
fitted_E0_quat, fitted_alpha_quat = popt_quat
perr_quat = np.sqrt(np.diag(pcov_quat))
print("\n=== Quaternionic Model Fit Results ===")
print(f"Fitted E0 = {fitted_E0_quat:.2f} ± {perr_quat[0]:.2f} GeV, Fitted alpha = {fitted_alpha_quat:.2f} ± {perr_quat[1]:.2f}")
model_quat_vals = model_quaternionic(bin_centers, fitted_E0_quat, fitted_alpha_quat)
chi2_quat = np.sum(((density - model_quat_vals) / errors)**2)
dof_quat = len(bin_centers) - 2  # 2 free parameters
AIC_quat = chi2_quat + 2 * 2
BIC_quat = chi2_quat + 2 * np.log(len(bin_centers))
print(f"chi2 = {chi2_quat:.2f}, degrees of freedom = {dof_quat}, AIC = {AIC_quat:.2f}, BIC = {BIC_quat:.2f}\n")

# =============================================================================
# STEP 5: Visualize the Data and Fit Results for Model Comparison
# =============================================================================
# Generate smooth curves for plotting
ET_fit = np.linspace(0, 500, 200)
pdf_LCDM = model_LCDM(ET_fit, fitted_E0_LCDM)
pdf_MOND = model_MOND(ET_fit, fitted_E0_MOND)
pdf_quat = model_quaternionic(ET_fit, fitted_E0_quat, fitted_alpha_quat)

plt.figure(figsize=(10, 7))
plt.errorbar(bin_centers, density, yerr=errors, fmt='o', label='Experimental Data (MET)', color='black')
plt.plot(ET_fit, pdf_LCDM, label='ΛCDM Model Fit', lw=2)
plt.plot(ET_fit, pdf_MOND, label='MOND Model Fit', lw=2)
plt.plot(ET_fit, pdf_quat, label='Quaternionic Model Fit', lw=2)
plt.xlabel('Missing Transverse Energy (GeV)')
plt.ylabel('Probability Density')
plt.title('Comparison of Public MET Data with Different Model Fits')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters (from CMS fit)
alpha = 0.035
E1 = 250  # GeV

# Energy range
ET = np.linspace(0, 1000, 500)

# Analytic template: alpha*(ET/E1)*sin^2(ET/E1)
epsilon_template = alpha * (ET/E1) * np.sin(ET/E1)**2

# Numerical solution (mocked here as ~5% deviation for illustration)
epsilon_numeric = epsilon_template * (1 + 0.05*np.sin(ET/50))

# Plotting
plt.figure(figsize=(8,5))
plt.plot(ET, epsilon_numeric, label='Numerical Solution', color='blue')
plt.plot(ET, epsilon_template, label='Analytic Template', color='orange', linestyle='dashed')
plt.xlabel("MET $ET$ (GeV)")
plt.ylabel(r"$\epsilon(ET)$")
plt.title("High-Energy Flux Comparison")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("epsilon_ET_sim.pdf")  # Output figure
plt.show()